# import necessary library

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# load dataset

In [29]:
df = pd.read_csv('Restaurant-Reviews.tsv', sep='\t', quoting=3)

# exploratory data analysis

In [30]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [31]:
df.tail()

,Review,Liked
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0
999,"Then, as if I hadn't wasted enough of my life ...",0


In [32]:
df.isna().sum()

Review    0
Liked     0
dtype: int64

In [33]:
print(len(df))
print(len(df[df['Liked'] == 1]))

1000
500


# data cleaning

In [34]:
import re
import nltk

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yandiher\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
from nltk.corpus import stopwords

In [37]:
df['Review'][0]

'Wow... Loved this place.'

In [38]:
review = re.sub(pattern='[^a-zA-Z]',
                repl=' ',
                string=df['Review'][0]).lower().split()

In [39]:
preview = []
for word in review:
    if word not in stopwords.words('english'):
        preview.append(word)
print(preview)
print(review)

['wow', 'loved', 'place']
['wow', 'loved', 'this', 'place']


In [48]:
review = [word for word in review if word not in stopwords.words(fileids='english')]

In [49]:
from nltk.stem import PorterStemmer
pStemmer = PorterStemmer()

In [51]:
review = [pStemmer.stem(word=word) for word in review]

In [53]:
review = ' '.join(review)
review

'wow love place'